In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16


In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
batch_size = 32

['.config', 'drive', 'sample_data']

Data prepration
reading all the subfolders in the main folder and puttig them together

In [16]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

data_dir = '/content/drive/MyDrive/training_data_v2'
img_height = 224
img_width = 224
data = []
labels = []
total_labels = os.listdir(data_dir)
print(os.listdir(data_dir))
for label in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, label)):
        for filename in os.listdir(os.path.join(data_dir, label)):
            img_path = os.path.join(data_dir, label, filename)
            img = keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
            img_array = keras.preprocessing.image.img_to_array(img) / 255.0
            data.append(img_array)
            labels.append(label)



['leftRear', 'rightRear', 'leftFront', 'front', 'rightFront', 'rear', 'other']


One hot encoding of labels

In [17]:
from tensorflow.keras.utils import to_categorical

mapping = {}
for x in range(len(total_labels)):
  mapping[total_labels[x]] = x

for x in range(len(labels)):
  labels[x] = mapping[labels[x]]

labels = to_categorical(labels)


In [20]:
# Split training data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    np.array(data), np.array(labels), test_size=0.2, random_state=25)


# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=2)

In [21]:
train_datagen = ImageDataGenerator(
    rotation_range=2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(
    X_train, y_train, batch_size=batch_size)

test_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()


test_generator = test_datagen.flow(
    X_test, y_test, batch_size=1)

val_generator = val_datagen.flow(
    X_val, y_val, batch_size=batch_size)

In [ ]:
# # Load pre-trained ResNet50 model
# base_model = keras.applications.ResNet50(
#     include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
# # Freeze model weights
# for layer in base_model.layers:
#     layer.trainable = False

In [22]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable=False


58889256/58889256 [==============================] - 2s 0us/step


In [23]:
# # Add new classifier layers
# x = layers.Flatten()(base_model.output)
# x = layers.Dense(256, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(6, activation='softmax')(x)

# model = keras.Model(inputs=base_model.input, outputs=x)
# # model.summary()

model = tf.keras.Sequential([
        base_model,  
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Reshape((1, 1, 512)),
        layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2), padding='same'),
        # layers.BatchNormalization()
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(7, activation='softmax')
    ])

# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# create the head of the model
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# predictions = Dense(6, activation='softmax')(x)

# combine the base model and head
# model = Model(inputs=base_model.input, outputs=predictions)

In [30]:
early = tf.keras.callbacks.EarlyStopping( patience=10,
                                          min_delta=0.001,
                                          restore_best_weights=True)

import tensorflow as tf

class AdaptiveLearningRateCallback(tf.keras.callbacks.Callback):
    def __init__(self, initial_lr, factor, patience):
        super().__init__()
        self.initial_lr = initial_lr
        self.factor = factor
        self.patience = patience
        self.best_loss = float('inf')
        self.wait = 0
    
    def on_epoch_end(self, epoch, logs=None):
        # Check if validation loss has improved
        val_loss = logs.get('val_loss')
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                # Reduce learning rate by factor if validation loss doesn't improve
                lr = tf.keras.backend.get_value(self.model.optimizer.lr)
                new_lr = lr * self.factor
                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
                print(f'Reducing learning rate to {new_lr}.')
                self.wait = 0


In [25]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [31]:
factor = 0.1
patience = 3
initial_lr = 0.001
lr_callback = AdaptiveLearningRateCallback(initial_lr, factor, patience)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
num_epochs = 200
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    callbacks=[early,lr_callback]
)


In [ ]:

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)

print(f'Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}')


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
n = 0
b = 11
img = test_generator[b][0][n]*255
gt = test_generator[b][1][n]
print(total_labels)
print('truth:',np.array(total_labels)[np.argmax(gt)]) 
pred = model.predict(test_generator[b][0])[0]
print('pred:',np.array(total_labels)[np.argmax(pred)] , max(pred) , pred)
cv2_imshow(img)


In [ ]:
# model.save("vgg16_25_32_pool_f1_0_65.h5")

In [29]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to disk
with open('model_2.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
# tflite_model.predict(test_generator[b][0])[0])

import numpy as np
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path='/content/model_1.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = test_generator[b][0]
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)
predicted_class = np.argmax(output_data)
